In [1]:
import sys
import SimpleITK as sitk
import json
import glob
import os
from tqdm import tqdm
import numpy as np
import torch
import shutil
sys.path.append("/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/src/utils")
from revert_normalisation import get_ct_normalisation_values, revert_normalisation
from image_metrics import ImageMetrics, ImageMetricsCompute
import pandas as pd

In [2]:
# Load config file
config = json.load(open("config_330.json"))
config

{'dataset_id': 330,
 'dataset_data_name': 'synthrad2025_task1_MR_AB_resXL',
 'dataset_target_name': 'synthrad2025_task1_CT_AB_resXL',
 'data_root': '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB',
 'preprocessing_CT': 'CT',
 'preprocessing_MRI': 'MR',
 'preprocessing_mask': 'no mask'}

In [3]:
# invert normalisation
ct_plan_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id'] + 1}_{config['dataset_target_name']}/nnUNetResEncUNetPlans_80G.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
dataset_plan = "nnUNetResEncUNetPlans_80G"
fold = 0
pred_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results/Dataset{config['dataset_id']}_{config['dataset_data_name']}/nnUNetTrainerMRCT__{dataset_plan}__3d_fullres/fold_{fold}/validation"
pred_path_revert_norm = pred_path + "_revert_norm"
revert_normalisation(pred_path, ct_mean, ct_std, save_path=pred_path_revert_norm)


CT mean: -660.54248046875, CT std: 452.00933837890625


  0%|          | 0/35 [00:00<?, ?it/s]

100%|██████████| 35/35 [00:28<00:00,  1.22it/s]


In [4]:
def result_analysis(pred_path, raw_data_path):
    """
    Analyze the results of the predictions.
    """
    pred_paths = sorted(glob.glob(os.path.join(pred_path, '*.mha')))
    gt_path = os.path.join(raw_data_path, "gt_segmentations")
    mask_path = os.path.join(raw_data_path, "masks")
    testing_metrics = ImageMetricsCompute()
    testing_metrics.init_storage(["mae", "psnr", "ms_ssim"])

    for pred_path in tqdm(pred_paths):
        filename = os.path.basename(pred_path)
        gt_file = os.path.join(gt_path, filename)
        mask_file = os.path.join(mask_path, filename)

        img_pred = sitk.ReadImage(pred_path)
        img_gt = sitk.ReadImage(gt_file)
        img_mask = sitk.ReadImage(mask_file, sitk.sitkUInt8)

        array_pred = sitk.GetArrayFromImage(img_pred)
        array_gt = sitk.GetArrayFromImage(img_gt)
        array_mask = sitk.GetArrayFromImage(img_mask)

        res = testing_metrics.score_patient(array_gt, array_pred, array_mask)
        testing_metrics.add(res, filename)

    # aggregate results
    results = testing_metrics.aggregate()

    df = pd.DataFrame(
            {
                'patient_id': testing_metrics.storage_id,
                'mae': testing_metrics.storage['mae'],
                'ms_ssim': testing_metrics.storage['ms_ssim'],
                'psnr': testing_metrics.storage['psnr'],
            }
        )

    # save results, and df in the folder "results"
    results_path = os.path.join(pred_path_revert_norm, "results.json")
    df_path = os.path.join(pred_path_revert_norm, "results_individual.csv")
    os.makedirs(os.path.dirname(results_path), exist_ok=True)
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=4)
    df.to_csv(df_path, index=False)
    return results, df

# raw_data_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id']}_{config['dataset_data_name']}"
raw_data_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id']}_{config['dataset_data_name']}"
results, df = result_analysis(pred_path_revert_norm, raw_data_path)
print("mean mae:", results['mae']['mean'])
print("mean psnr:", results['psnr']['mean'])
print("mean ms_ssim:", results['ms_ssim']['mean'])

100%|██████████| 35/35 [05:23<00:00,  9.23s/it]

mean mae: 105.57495792399222
mean psnr: 25.332086495459862
mean ms_ssim: 0.7779173987887936


In [6]:
# save results, and df in the folder "results"
results_path = os.path.join(pred_path_revert_norm, "results.json")
df_path = os.path.join(pred_path_revert_norm, "results_df.csv")
os.makedirs(os.path.dirname(results_path), exist_ok=True)
with open(results_path, 'w') as f:
    json.dump(results, f, indent=4)
df.to_csv(df_path, index=False)


# Dataset 302

In [4]:
# Load config file
config = json.load(open("config_302.json"))

ct_plan_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id'] + 1}_{config['dataset_target_name']}/nnUNetPlans.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
pred_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results/Dataset{config['dataset_id']}_{config['dataset_data_name']}/nnUNetTrainerMRCT__nnUNetPlans__3d_fullres/fold_0/validation"
pred_path_revert_norm = pred_path + "_revert_norm"
revert_normalisation(pred_path, ct_mean, ct_std, save_path=pred_path_revert_norm)


CT mean: -756.8418579101562, CT std: 460.713623046875


100%|██████████| 45/45 [00:17<00:00,  2.59it/s]


In [5]:
raw_data_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id']}_{config['dataset_data_name']}"
results, df = result_analysis(pred_path_revert_norm, raw_data_path)
print("mean mae:", results['mae']['mean'])
print("mean psnr:", results['psnr']['mean'])
print("mean ms_ssim:", results['ms_ssim']['mean'])

100%|██████████| 45/45 [03:37<00:00,  4.84s/it]

mean mae: 113.70820987479676
mean psnr: 25.445675841563286
mean ms_ssim: 0.8954014513641562


In [6]:
# save results, and df in the folder "results"
results_path = os.path.join(pred_path_revert_norm, "results.json")
df_path = os.path.join(pred_path_revert_norm, "results_df.csv")
os.makedirs(os.path.dirname(results_path), exist_ok=True)
with open(results_path, 'w') as f:
    json.dump(results, f, indent=4)
df.to_csv(df_path, index=False)


# Dataset 304

In [7]:
# Load config file
config = json.load(open("config_304.json"))

ct_plan_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id'] + 1}_{config['dataset_target_name']}/nnUNetPlans.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
pred_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results/Dataset{config['dataset_id']}_{config['dataset_data_name']}/nnUNetTrainerMRCT__nnUNetPlans__3d_fullres/fold_0/validation"
pred_path_revert_norm = pred_path + "_revert_norm"
revert_normalisation(pred_path, ct_mean, ct_std, save_path=pred_path_revert_norm)


CT mean: -753.1259765625, CT std: 409.43475341796875


 70%|███████   | 26/37 [00:41<00:25,  2.29s/it]

100%|██████████| 37/37 [01:04<00:00,  1.73s/it]


In [8]:
raw_data_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id']}_{config['dataset_data_name']}"
results, df = result_analysis(pred_path_revert_norm, raw_data_path)
print("mean mae:", results['mae']['mean'])
print("mean psnr:", results['psnr']['mean'])
print("mean ms_ssim:", results['ms_ssim']['mean'])

100%|██████████| 37/37 [06:40<00:00, 10.83s/it]

mean mae: 110.81249830293514
mean psnr: 25.484432741206287
mean ms_ssim: 0.8293993379400778


In [9]:
# save results, and df in the folder "results"
results_path = os.path.join(pred_path_revert_norm, "results.json")
df_path = os.path.join(pred_path_revert_norm, "results_df.csv")
os.makedirs(os.path.dirname(results_path), exist_ok=True)
with open(results_path, 'w') as f:
    json.dump(results, f, indent=4)
df.to_csv(df_path, index=False)
